### Build the training\evaluation dataset

In [1]:
import yfinance
import pandas_ta #noqa

TICKER = 'PETR4.SA' # TODO: replace this with your own ticker

TRAIN_START_DATE = '2021-02-09' # TODO: replace this with your own start date
TRAIN_END_DATE = '2021-09-30' # TODO: replace this with your own end date
EVAL_START_DATE = '2021-10-01' # TODO: replace this with your own end date
EVAL_END_DATE = '2021-11-12' # TODO: replace this with your own end date
yf_ticker = yfinance.Ticker(ticker=TICKER)

df_training = yf_ticker.history(start=TRAIN_START_DATE, end=TRAIN_END_DATE, interval='60m')
df_training.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
df_training["Volume"] = df_training["Volume"].astype(int)
df_training.ta.log_return(append=True, length=16)
df_training.ta.rsi(append=True, length=14)
df_training.ta.macd(append=True, fast=12, slow=26)
df_training.dropna().to_csv('training.csv', index=True)

df_evaluation = yf_ticker.history(start=EVAL_START_DATE, end=EVAL_END_DATE, interval='60m')
df_evaluation.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
df_evaluation["Volume"] = df_evaluation["Volume"].astype(int)
df_evaluation.ta.log_return(append=True, length=16)
df_evaluation.ta.rsi(append=True, length=14)
df_evaluation.ta.macd(append=True, fast=12, slow=26)
df_evaluation.dropna().to_csv('evaluation.csv', index=True)

### Create the environment build function

In [2]:
import pandas as pd
from tensortrade.feed.core import DataFeed, Stream
from tensortrade.oms.instruments import Instrument
from tensortrade.oms.exchanges import Exchange, ExchangeOptions
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.wallets import Wallet, Portfolio
import tensortrade.env.default as default

In [3]:
def create_env(config):
    dataset = pd.read_csv(filepath_or_buffer=config["csv_filename"], parse_dates=['Datetime']).fillna(method='backfill').fillna(method='ffill')
    ttse_commission = 0.0035 # TODO: adjust according to your commission percentage, if present
    price = Stream.source(list(dataset["Close"]), dtype="float").rename("USD-TTRD")
    ttse_options = ExchangeOptions(commission=ttse_commission)
    ttse_exchange = Exchange("TTSE", service=execute_order, options=ttse_options)(price)
        
    # Instruments, Wallets and Portfolio
    USD = Instrument("USD", 2, "US Dollar")
    TTRD = Instrument("TTRD", 2, "TensorTrade Corp")
    cash = Wallet(ttse_exchange, 1000 * USD) # This is the starting cash we are going to use
    asset = Wallet(ttse_exchange, 0 * TTRD) # And we will start owning 0 stocks of TTRD
    portfolio = Portfolio(USD, [cash, asset])
    
    # Renderer feed
    renderer_feed = DataFeed([
        Stream.source(list(dataset["Datetime"])).rename("date"),
        Stream.source(list(dataset["Open"]), dtype="float").rename("open"),
        Stream.source(list(dataset["High"]), dtype="float").rename("high"),
        Stream.source(list(dataset["Low"]), dtype="float").rename("low"),
        Stream.source(list(dataset["Close"]), dtype="float").rename("close"),
        Stream.source(list(dataset["Volume"]), dtype="float").rename("volume")
    ])
    
    features = []
    
    for c in dataset.columns[1:]:
        s = Stream.source(list(dataset[c]), dtype="float").rename(dataset[c].name)
        features += [s]
        
    feed = DataFeed(features)
    feed.compile()
    reward_scheme = default.rewards.SimpleProfit(window_size=config["reward_window_size"])
    action_scheme = default.actions.BSH(cash=cash, asset=asset)
    env = default.create(
        feed=feed,
        portfolio=portfolio,
        action_scheme=action_scheme,
        reward_scheme=reward_scheme,
        renderer_feed=renderer_feed,
        renderer=[],
        window_size=config["window_size"],
        max_allowed_loss=config["max_allowed_loss"]
    )
    return env

### Initialize and run Ray


In [4]:
import ray
from ray import tune
from ray.tune.registry import register_env

# Let's define some tuning parameters
FC_SIZE = tune.grid_search([[256, 256]]) # Those are the alternatives that ray.tune will try...
LEARNING_RATE = tune.grid_search([0.0005]) # ... and they will be combined with these ones ...
MINIBATCH_SIZE = tune.grid_search([5]) # ... and these ones, in a cartesian product.

# Initialize Ray
ray.init() # There are *LOTS* of initialization parameters, like specifying the maximum number of CPUs\GPUs to allocate. 
# For now just leave it alone.

# Register our environment, specifying which is the environment creation function
register_env("MyTrainingEnv", create_env)

# Specific configuration keys that will be used during training
env_config_training = {
    "window_size": 14, # We want to look at the last 14 samples (hours)
    "reward_window_size": 7, # And calculate reward based on the actions taken in the next 7 hours
    "max_allowed_loss": 0.10, # If it goes past 10% loss during the iteration, we don't want to waste time on a "loser".
    "csv_filename": "C:\\Users\\mathe\\Desktop\\Desktop\\Estudos\\Courses\\(Framework) TensorTrade\\training.csv" # The variable that will be used to differentiate training and validation datasets
}

# Specific configuration keys that will be used during evaluation (only the overridden ones)
env_config_evaluation = {
    "max_allowed_loss": 1.00, # During validation runs we want to see how bad it would go. Even up to 100% loss.
    "csv_filename": "C:\\Users\\mathe\\Desktop\\Desktop\\Estudos\\Courses\\(Framework) TensorTrade\\evaluation.csv", # The variable that will be used to differentiate training and validation datasets
}

analysis = tune.run(
    run_or_experiment="PPO", # We'll be using the builtin PPO agent in RLLib
    name="MyExperiment1",
    metric='episode_reward_mean',
    mode='max',
    stop={
        "training_iteration": 5 # Let's do 5 steps for each hyperparameter combination
    },
    config={
        "env": "MyTrainingEnv",
        "env_config": env_config_training, # The dictionary we built before
        "log_level": "WARNING",
        "framework": "tf2",
        "ignore_worker_failures": True,
        "num_workers": 1, # One worker per agent. You can increase this but it will run fewer parallel trainings.
        "num_envs_per_worker": 1,
        "num_gpus": 1, # I yet have to understand if using a GPU is worth it, for our purposes, but I think it's not. This way you can train on a non-gpu enabled system.
        "clip_rewards": True,
        "lr": LEARNING_RATE, # Hyperparameter grid search defined above
        "gamma": 0.50, # This can have a big impact on the result and needs to be properly tuned (range is 0 to 1)
        "observation_filter": "MeanStdFilter",
        "model": {
            "fcnet_hiddens": FC_SIZE, # Hyperparameter grid search defined above
        },
        "sgd_minibatch_size": MINIBATCH_SIZE, # Hyperparameter grid search defined above
        "evaluation_interval": 1, # Run evaluation on every iteration
        "evaluation_config": {
            "env_config": env_config_evaluation, # The dictionary we built before (only the overriding keys to use in evaluation)
            "explore": False, # We don't want to explore during evaluation. All actions have to be repeatable.
            },
        },
        num_samples=1, # Have one sample for each hyperparameter combination. You can have more to average out randomness.
        keep_checkpoints_num=10, # Keep the last 2 checkpoints
        checkpoint_freq=1, # Do a checkpoint on each iteration (slower but you can pick more finely the checkpoint to use later)
)

2022-01-08 16:53:18,641	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2022-01-08 16:53:32,340	ERROR syncer.py:111 -- Log sync requires rsync to be installed.
 pid=11752) 2022-01-08 16:53:43,703	INFO trainer.py:712 -- Executing eagerly (framework='tf2'), with eager_tracing=False. For production workloads, make sure to set `eager_tracing=True` in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
 pid=11752) 2022-01-08 16:53:57,182	WARNING deprecation.py:45 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!


Trial name,status,loc,lr,model/fcnet_hiddens,sgd_minibatch_size
PPO_MyTrainingEnv_a6a7a_00000,RUNNING,127.0.0.1:11752,0.0005,"[256, 256]",5


 pid=11752) 2022-01-08 16:53:59,845	INFO trainable.py:124 -- Trainable.setup took 16.842 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
 pid=11752) 2022-01-08 16:53:59,846	WARNING util.py:57 -- Install gputil for GPU system monitoring.


Trial name,status,loc,lr,model/fcnet_hiddens,sgd_minibatch_size
PPO_MyTrainingEnv_a6a7a_00000,RUNNING,127.0.0.1:11752,0.0005,"[256, 256]",5


Trial name,status,loc,lr,model/fcnet_hiddens,sgd_minibatch_size
PPO_MyTrainingEnv_a6a7a_00000,RUNNING,127.0.0.1:11752,0.0005,"[256, 256]",5


...

Trial name,status,loc,lr,model/fcnet_hiddens,sgd_minibatch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MyTrainingEnv_a6a7a_00000,RUNNING,127.0.0.1:11752,0.0005,"[256, 256]",5,4,2822.28,16000,-0.580606,-0.355717,-0.702222,146.34


Trial name,status,loc,lr,model/fcnet_hiddens,sgd_minibatch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MyTrainingEnv_a6a7a_00000,RUNNING,127.0.0.1:11752,0.0005,"[256, 256]",5,4,2822.28,16000,-0.580606,-0.355717,-0.702222,146.34


Result for PPO_MyTrainingEnv_a6a7a_00000:
  agent_timesteps_total: 20000
  custom_metrics: {}
  date: 2022-01-08_17-52-53
  done: true
  episode_len_mean: 178.99
  episode_media: {}
  episode_reward_max: -0.35571659678320633
  episode_reward_mean: -0.5800754445519195
  episode_reward_min: -0.7022222719904686
  episodes_this_iter: 9
  episodes_total: 124
  evaluation:
    custom_metrics: {}
    episode_len_mean: 143.0
    episode_media: {}
    episode_reward_max: -0.6953121599729144
    episode_reward_mean: -0.9645586438902448
    episode_reward_min: -1.1564998335885663
    episodes_this_iter: 10
    hist_stats:
      episode_lengths:
      - 143
      - 143
      - 143
      - 143
      - 143
      - 143
      - 143
      - 143
      - 143
      - 143
      episode_reward:
      - -0.6953121599729144
      - -0.7867089014020817
      - -0.9075522034144284
      - -1.0158647069388445
      - -0.9949137783633113
      - -0.9470556817775354
      - -1.0008179040784615
      - -1.049947257

Trial name,status,loc,lr,model/fcnet_hiddens,sgd_minibatch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MyTrainingEnv_a6a7a_00000,TERMINATED,127.0.0.1:11752,0.0005,"[256, 256]",5,5,3533.86,20000,-0.580075,-0.355717,-0.702222,178.99


2022-01-08 17:52:54,180	INFO tune.py:626 -- Total run time: 3564.81 seconds (3561.90 seconds for the tuning loop).


In [5]:
import ta
import pandas as pd
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC
from tensortrade.oms.wallets import Wallet, Portfolio
%matplotlib inline

 pid=13840) [2022-01-08 17:52:54,117 C 13840 4324] core_worker.cc:835:  Check failed: _s.ok() Bad status: IOError: Unknown error
 pid=13840) *** StackTrace Information ***
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyInit__raylet
 pid=13840)     PyList_New
 pid=13840)     Py_CheckFunctionResult
 pid=13840)     PyEval_EvalFrameDefault
 pid=13840)     Py_CheckFunctionResult
 pid=13840)     PyEval_EvalFrameDefault
 pid=13840)     PyEval_EvalCodeWithName
 pid=13840)     PyEval_EvalCodeEx
 pid=13840)     PyEval_EvalCode
 pid=13840

ModuleNotFoundError: No module named 'ta'